# Convolutiona Neural Network

## The task is to recognize if there is a dog or a cat on the given image

# 1. Building the CNN

### Importing keras libraries

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

Using TensorFlow backend.


In [7]:
classifier = Sequential()

### Convolution layer - using filters

In [8]:
classifier.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))

### Pooling layer - using maxpooling

In [9]:
classifier.add(MaxPooling2D(pool_size=(2,2)))

### Flattening layer - creating vector instead of array structure

In [10]:
classifier.add(Flatten())

### Fully connection layer

In [11]:
classifier.add(Dense(units=128,activation='relu'))

In [12]:
classifier.add(Dense(units=1,activation='sigmoid'))

### Compiling CNN

In [14]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 2. Training CNN

### Preprocessing

In [15]:
from keras.preprocessing.image import ImageDataGenerator

In [20]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')



Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [30]:
from keras.callbacks import EarlyStopping

In [31]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

In [32]:
from keras.callbacks import ModelCheckpoint

In [33]:
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

In [34]:
with tf.device('/gpu:0'):
    classifier.fit_generator(training_set,
                         steps_per_epoch = 8000,
                         epochs = 4,
                         validation_data = test_set,
                         validation_steps = 2000,
                             callbacks=[es,mc]
                        )

Epoch 1/4
8000/8000 [==============================] - 870s 109ms/step - loss: 0.1822 - acc: 0.9254 - val_loss: 1.0778 - val_acc: 0.7474
Epoch 2/4
8000/8000 [==============================] - 880s 110ms/step - loss: 0.1594 - acc: 0.9358 - val_loss: 1.1589 - val_acc: 0.7416
Epoch 00002: early stopping


So now let's test our model

Let's load our saved weights

In [17]:
classifier.load_weights('best_model.h5')

Here is the picture of our dog we want our model to classify as dog

![Title of a dog](dataset/individual/Mixed/dog.4056.jpg)

In [18]:
ind_gen = ImageDataGenerator(rescale = 1./255).flow_from_directory(
                            'dataset/individual'
    ,target_size=(64,64)
    ,class_mode=None
    ,batch_size=32
    ,shuffle=False
)

Found 1 images belonging to 1 classes.


In [19]:
ind_gen.reset()

In [20]:
prediction = classifier.predict_generator(ind_gen, steps=1)

In [25]:
def dogorcat(probability):
    if(probability>0.5):
        return "dog"
    else:
        return "cat"

In [26]:
print(dogorcat(prediction))

dog
